In [2]:

import argparse, os, platform, numpy as np, matplotlib.pyplot as plt
from typing import Tuple
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, classification_report

#Args
def get_args():
    p = argparse.ArgumentParser()
    p.add_argument("--dataset", type=str, default="mnist",
                   choices=["mnist","emnist-letters"],
                   help="Choose MNIST (10 classes) or EMNIST letters (26 classes)")
    p.add_argument("--epochs", type=int, default=6)
    p.add_argument("--batch_size", type=int, default=128)
    p.add_argument("--lr", type=float, default=1e-3)
    p.add_argument("--augment", action="store_true", help="Apply light augmentation")
    p.add_argument("--out", type=str, default="outputs_task3")
    # In Colab, we parse known args and ignore the rest
    return p.parse_known_args()[0]

#Data loading
def get_transforms(augment: bool):
    base = [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    if augment:
        aug = [transforms.RandomAffine(degrees=8, translate=(0.08,0.08), scale=(0.95,1.05))]
        return transforms.Compose(aug + base)
    return transforms.Compose(base)

def load_data(dataset: str, batch_size: int, augment: bool, pin_memory: bool, num_workers: int) -> Tuple[DataLoader, DataLoader, int]:
    root = "./data"
    if dataset == "mnist":
        num_classes = 10
        train_ds = datasets.MNIST(root, train=True, download=True, transform=get_transforms(augment))
        test_ds  = datasets.MNIST(root, train=False, download=True, transform=get_transforms(False))
    else:  # emnist-letters
        # labels can be 1..26 or remapped to 0..25 depending on torchvision version
        num_classes = 26
        train_ds = datasets.EMNIST(root, split="letters", train=True, download=True, transform=get_transforms(augment))
        test_ds  = datasets.EMNIST(root, split="letters", train=False, download=True, transform=get_transforms(False))

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=pin_memory
    )
    test_loader  = DataLoader(
        test_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=pin_memory
    )
    return train_loader, test_loader, num_classes

#Model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 28x28 -> 14x14
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # 14x14 -> 7x7
            nn.Flatten(),
            nn.Linear(64*7*7, 128), nn.ReLU(), nn.Dropout(0.25),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

#Train / Evaluate
def train_one_epoch(model, loader, criterion, opt, device):
    model.train()
    total, correct, loss_sum = 0, 0, 0.0
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        opt.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        opt.step()
        loss_sum += loss.item() * x.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return loss_sum/total, correct/total

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    y_true, y_pred = [], []
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        loss_sum += loss.item() * x.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
        y_true.extend(y.cpu().numpy().tolist())
        y_pred.extend(pred.cpu().numpy().tolist())
    return loss_sum/total, correct/total, np.array(y_true), np.array(y_pred)

def plot_confusion(cm, class_names, path):
    fig, ax = plt.subplots(figsize=(6,6))
    im = ax.imshow(cm, cmap="Blues")
    ax.set_title("Confusion Matrix")
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_xticks(range(len(class_names))); ax.set_yticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=90); ax.set_yticklabels(class_names)
    for (i,j),v in np.ndenumerate(cm):
        ax.text(j, i, str(v), ha="center", va="center", fontsize=7)
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout(); plt.savefig(path, dpi=150); plt.close(fig)

#Main
def main():
    args = get_args()
    os.makedirs(args.out, exist_ok=True)

    # Device + DataLoader settings (fixes your warning)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_pin_memory = (device.type == "cuda")  # only pin if GPU is present
    # Windows often behaves better with 0 workers inside PyCharm
    is_windows = (platform.system().lower() == "windows")
    num_workers = 0 if is_windows else 2

    print(f"Using device: {device} | pin_memory={use_pin_memory} | num_workers={num_workers}")

    train_loader, test_loader, num_classes = load_data(
        args.dataset, args.batch_size, args.augment, use_pin_memory, num_workers
    )

    model = SimpleCNN(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=args.lr)

    best_acc, best_path = 0.0, os.path.join(args.out, f"{args.dataset}_best.pt")

    for epoch in range(1, args.epochs+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, criterion, opt, device)
        te_loss, te_acc, y_true, y_pred = evaluate(model, test_loader, criterion, device)
        if te_acc > best_acc:
            best_acc = te_acc
            torch.save(model.state_dict(), best_path)
        print(f"Epoch {epoch:02d} | train_loss {tr_loss:.4f} acc {tr_acc:.4f} | "
              f"val_loss {te_loss:.4f} acc {te_acc:.4f}")

    print(f"Best val accuracy: {best_acc:.4f}; saved: {best_path}")

    # Reload best for final reporting
    model.load_state_dict(torch.load(best_path, map_location=device))
    _, _, y_true, y_pred = evaluate(model, test_loader, criterion, device)

    # Build class names robustly based on labels we actually saw
    classes = np.unique(np.concatenate([y_true, y_pred]))
    if args.dataset == "mnist":
        class_names = [str(int(c)) for c in classes]
    else:
        # EMNIST letters can be 0..25 or 1..26; map to a..z accordingly
        if classes.min() == 1:  # 1..26
            class_names = [chr(ord('a') + int(c) - 1) for c in classes]
        else:  # 0..25
            class_names = [chr(ord('a') + int(c)) for c in classes]

    print("\nClassification report (macro F1 included):")
    print(classification_report(y_true, y_pred, labels=classes, target_names=class_names, digits=4))

    cm = confusion_matrix(y_true, y_pred, labels=classes)
    cm_path = os.path.join(args.out, f"{args.dataset}_confusion_matrix.png")
    plot_confusion(cm, class_names, cm_path)
    print(f"Saved confusion matrix → {cm_path}")

# Call the main function directly in the Colab cell or as a script
if __name__ == "__main__":
    main()

Using device: cpu | pin_memory=False | num_workers=2


100%|██████████| 9.91M/9.91M [00:00<00:00, 103MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 39.5MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 35.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.36MB/s]


Epoch 01 | train_loss 0.1981 acc 0.9396 | val_loss 0.0405 acc 0.9875
Epoch 02 | train_loss 0.0582 acc 0.9823 | val_loss 0.0413 acc 0.9855
Epoch 03 | train_loss 0.0443 acc 0.9860 | val_loss 0.0306 acc 0.9900
Epoch 04 | train_loss 0.0340 acc 0.9896 | val_loss 0.0268 acc 0.9905
Epoch 05 | train_loss 0.0274 acc 0.9913 | val_loss 0.0236 acc 0.9927
Epoch 06 | train_loss 0.0224 acc 0.9925 | val_loss 0.0247 acc 0.9925
Best val accuracy: 0.9927; saved: outputs_task3/mnist_best.pt

Classification report (macro F1 included):
              precision    recall  f1-score   support

           0     0.9889    0.9980    0.9934       980
           1     0.9965    1.0000    0.9982      1135
           2     0.9866    0.9961    0.9913      1032
           3     0.9970    0.9921    0.9945      1010
           4     0.9959    0.9929    0.9944       982
           5     0.9866    0.9933    0.9899       892
           6     0.9968    0.9854    0.9911       958
           7     0.9961    0.9854    0.9907    